In [0]:
from pyspark.sql.types import StringType, MapType
mapCol = MapType(StringType(),StringType(),False)

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, MapType
schema = StructType([
    StructField('name', StringType(), True),
    StructField('properties', MapType(StringType(),StringType()),True)
])

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySpark').getOrCreate()
dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'grey','eye':'grey'}),
        ('Jefferson',{'hair':'brown','eye':''})
        ]
df = spark.createDataFrame(data=dataDictionary, schema = schema)
df.printSchema()
df.show(truncate=False)

root
-- name: string (nullable = true)
-- properties: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
name |properties |
+----------+-----------------------------+
James |{eye -> brown, hair -> black}|
Michael |{eye -> null, hair -> brown} |
Robert |{eye -> black, hair -> red} |
Washington|{eye -> grey, hair -> grey} |
Jefferson |{eye -> , hair -> brown} |
+----------+-----------------------------+

In [0]:
df3=df.rdd.map(lambda x: \
    (x.name,x.properties["hair"],x.properties["eye"])) \
    .toDF(["name","hair","eye"])
df3.printSchema()
df3.show()


root
-- name: string (nullable = true)
-- hair: string (nullable = true)
-- eye: string (nullable = true)

+----------+-----+-----+
 name| hair| eye|
+----------+-----+-----+
 James|black|brown|
 Michael|brown| null|
 Robert| red|black|
Washington| grey| grey|
 Jefferson|brown| |
+----------+-----+-----+

In [0]:
df.withColumn("hair",df.properties.getItem("hair")) \
  .withColumn("eye",df.properties.getItem("eye")) \
  .drop("properties") \
  .show()
#OR
df.withColumn("hair",df.properties["hair"]) \
  .withColumn("eye",df.properties["eye"]) \
  .drop("properties") \
  .show()

+----------+-----+-----+
 name| hair| eye|
+----------+-----+-----+
 James|black|brown|
 Michael|brown| null|
 Robert| red|black|
Washington| grey| grey|
 Jefferson|brown| |
+----------+-----+-----+

+----------+-----+-----+
 name| hair| eye|
+----------+-----+-----+
 James|black|brown|
 Michael|brown| null|
 Robert| red|black|
Washington| grey| grey|
 Jefferson|brown| |
+----------+-----+-----+

In [0]:
from pyspark.sql.functions import explode
df.select(df.name,explode(df.properties)).show()

+----------+----+-----+
 name| key|value|
+----------+----+-----+
 James| eye|brown|
 James|hair|black|
 Michael| eye| null|
 Michael|hair|brown|
 Robert| eye|black|
 Robert|hair| red|
Washington| eye| grey|
Washington|hair| grey|
 Jefferson| eye| |
 Jefferson|hair|brown|
+----------+----+-----+

In [0]:
from pyspark.sql.functions import map_keys
df.select(df.name,map_keys(df.properties)).show()

+----------+--------------------+
 name|map_keys(properties)|
+----------+--------------------+
 James| [eye, hair]|
 Michael| [eye, hair]|
 Robert| [eye, hair]|
Washington| [eye, hair]|
 Jefferson| [eye, hair]|
+----------+--------------------+

In [0]:
from pyspark.sql.functions import explode,map_keys
keysDF = df.select(explode(map_keys(df.properties))).distinct()
keysList = keysDF.rdd.map(lambda x:x[0]).collect()
print(keysList)

['eye', 'hair']

In [0]:
from pyspark.sql.functions import map_values
df.select(df.name,map_values(df.properties)).show()

+----------+----------------------+
 name|map_values(properties)|
+----------+----------------------+
 James| [brown, black]|
 Michael| [ null, brown]|
 Robert| [black, red]|
Washington| [grey, grey]|
 Jefferson| [, brown]|
+----------+----------------------+

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySpark').getOrCreate()

dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'grey','eye':'grey'}),
        ('Jefferson',{'hair':'brown','eye':''})
        ]
df = spark.createDataFrame(data=dataDictionary, schema = ['name','properties'])
df.printSchema()
df.show(truncate=False)

root
-- name: string (nullable = true)
-- properties: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
name |properties |
+----------+-----------------------------+
James |{eye -> brown, hair -> black}|
Michael |{eye -> null, hair -> brown} |
Robert |{eye -> black, hair -> red} |
Washington|{eye -> grey, hair -> grey} |
Jefferson |{eye -> , hair -> brown} |
+----------+-----------------------------+

In [0]:
df3=df.rdd.map(lambda x: \
    (x.name,x.properties["hair"],x.properties["eye"])) \
    .toDF(["name","hair","eye"])
df3.printSchema()
df3.show()

root
-- name: string (nullable = true)
-- hair: string (nullable = true)
-- eye: string (nullable = true)

+----------+-----+-----+
 name| hair| eye|
+----------+-----+-----+
 James|black|brown|
 Michael|brown| null|
 Robert| red|black|
Washington| grey| grey|
 Jefferson|brown| |
+----------+-----+-----+

In [0]:
df.withColumn("hair",df.properties.getItem("hair")) \
  .withColumn("eye",df.properties.getItem("eye")) \
  .drop("properties") \
  .show()

+----------+-----+-----+
 name| hair| eye|
+----------+-----+-----+
 James|black|brown|
 Michael|brown| null|
 Robert| red|black|
Washington| grey| grey|
 Jefferson|brown| |
+----------+-----+-----+

In [0]:
df.withColumn("hair",df.properties["hair"]) \
  .withColumn("eye",df.properties["eye"]) \
  .drop("properties") \
  .show()

+----------+-----+-----+
 name| hair| eye|
+----------+-----+-----+
 James|black|brown|
 Michael|brown| null|
 Robert| red|black|
Washington| grey| grey|
 Jefferson|brown| |
+----------+-----+-----+

In [0]:
from pyspark.sql.functions import explode,map_keys,col
keysDF = df.select(explode(map_keys(df.properties))).distinct()
keysList = keysDF.rdd.map(lambda x:x[0]).collect()
keyCols = list(map(lambda x: col("properties").getItem(x).alias(str(x)), keysList))
df.select(df.name, *keyCols).show()

+----------+-----+-----+
 name| eye| hair|
+----------+-----+-----+
 James|brown|black|
 Michael| null|brown|
 Robert|black| red|
Washington| grey| grey|
 Jefferson| |brown|
+----------+-----+-----+

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

spark = SparkSession.builder.appName('PySpark').getOrCreate()
data = [ ("36636","Finance",3000,"USA"), 
    ("40288","Finance",5000,"Aus"), 
    ("42114","Sales",3900,"USA"), 
    ("39192","Marketing",2500,"CAN"), 
    ("34534","Sales",6500,"USA") ]
schema = StructType([
     StructField('id', StringType(), True),
     StructField('dept', StringType(), True),
     StructField('salary', IntegerType(), True),
     StructField('location', StringType(), True)
     ])

df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
-- id: string (nullable = true)
-- dept: string (nullable = true)
-- salary: integer (nullable = true)
-- location: string (nullable = true)

+-----+---------+------+--------+
id |dept |salary|location|
+-----+---------+------+--------+
36636|Finance |3000 |USA |
40288|Finance |5000 |Aus |
42114|Sales |3900 |USA |
39192|Marketing|2500 |CAN |
34534|Sales |6500 |USA |
+-----+---------+------+--------+

In [0]:
#Convert columns to Map
from pyspark.sql.functions import col,lit,create_map
df = df.withColumn("propertiesMap",create_map(
        lit("salary"),col("salary"),
        lit("location"),col("location")
        )).drop("salary","location")
df.printSchema()
df.show(truncate=False)

root
-- id: string (nullable = true)
-- dept: string (nullable = true)
-- propertiesMap: map (nullable = false)
 |-- key: string
 |-- value: string (valueContainsNull = true)

+-----+---------+---------------------------------+
id |dept |propertiesMap |
+-----+---------+---------------------------------+
36636|Finance |{salary -> 3000, location -> USA}|
40288|Finance |{salary -> 5000, location -> Aus}|
42114|Sales |{salary -> 3900, location -> USA}|
39192|Marketing|{salary -> 2500, location -> CAN}|
34534|Sales |{salary -> 6500, location -> USA}|
+-----+---------+---------------------------------+

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
spark = SparkSession.builder.appName('PySpark').getOrCreate()
data = [ ("36636","Finance",(3000,"USA")), 
    ("40288","Finance",(5000,"Aus")), 
    ("42114","Sales",(3900,"USA")), 
    ("39192","Marketing",(2500,"CAN")), 
    ("34534","Sales",(6500,"USA")) ]
schema = StructType([
     StructField('id', StringType(), True),
     StructField('dept', StringType(), True),
     StructField('properties', StructType([
         StructField('salary', IntegerType(), True),
         StructField('location', StringType(), True)
         ]))
     ])

df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
-- id: string (nullable = true)
-- dept: string (nullable = true)
-- properties: struct (nullable = true)
 |-- salary: integer (nullable = true)
 |-- location: string (nullable = true)

+-----+---------+-----------+
id |dept |properties |
+-----+---------+-----------+
36636|Finance |{3000, USA}|
40288|Finance |{5000, Aus}|
42114|Sales |{3900, USA}|
39192|Marketing|{2500, CAN}|
34534|Sales |{6500, USA}|
+-----+---------+-----------+

In [0]:
#Convert struct type to Map
from pyspark.sql.functions import col,lit,create_map
df = df.withColumn("propertiesMap",create_map(
        lit("salary"),col("properties.salary"),
        lit("location"),col("properties.location")
        )).drop("properties")
df.printSchema()
df.show(truncate=False)

root
-- id: string (nullable = true)
-- dept: string (nullable = true)
-- propertiesMap: map (nullable = false)
 |-- key: string
 |-- value: string (valueContainsNull = true)

+-----+---------+---------------------------------+
id |dept |propertiesMap |
+-----+---------+---------------------------------+
36636|Finance |{salary -> 3000, location -> USA}|
40288|Finance |{salary -> 5000, location -> Aus}|
42114|Sales |{salary -> 3900, location -> USA}|
39192|Marketing|{salary -> 2500, location -> CAN}|
34534|Sales |{salary -> 6500, location -> USA}|
+-----+---------+---------------------------------+